In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
import plotly.graph_objects as go

import seaborn as sns
TSLA = pd.read_csv("../input/tesla-stock-data-from-2010-to-2020/TSLA.csv")

In [1]:
TSLA['Date'] = pd.to_datetime(TSLA['Date'])
TSLA.index = TSLA['Date']
TSLA.head(3)

In [1]:
Tesla=TSLA.rename(columns={'Adj Close': 'AdjClose'})
Tesla.head(3)

In [1]:
sns.pairplot(Tesla[["Open", "High", "Close", "Volume"]], diag_kind="kde")


In [1]:
fig = px.line(Tesla, x='Date', y='Volume')
fig.show()

In [1]:
fig = go.Figure(data=[go.Candlestick(
    x=Tesla['Date'],
    open=Tesla['Open'], high=Tesla['High'],
    low=Tesla['Low'], close=Tesla['Close'],
    increasing_line_color= 'cyan', decreasing_line_color= 'green'
)])

fig.show()

# Tesla HPY (Holding Period Yield) Model

<img class="mntl-lightbox__img" src="https://www.investopedia.com/thmb/KKV8GggtRpTzoUSICB5jDEnXpII=/1280x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/HoldingPeriodReturn-2917125a133f40818f2e4005e291538c.jpg">

https://www.investopedia.com/terms/h/holdingperiodreturn-yield.asp

In [1]:
Tesla_HPY = pd.DataFrame({'Date':Tesla['Date'], 'HPY':Tesla['Close'] / Tesla['Open']-1})
Tesla_HPY.head(3)

In [1]:
# Telsa HPY static statement 
Tesla_HPY.describe()

In [1]:
# Tesla visualization 
fig = go.Figure([go.Scatter(x=Tesla_HPY['Date'], y=Tesla_HPY['HPY'])])
fig.show()

* # keras prediction modeling 

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder

In [1]:
TeslaDF = Tesla.loc[:,['AdjClose','Volume']]
TeslaDF['HPY'] = ((Tesla['Close'] / Tesla['Open'])-1) 
TeslaDF['HL_PCT'] = (Tesla['High'] - Tesla['Low'])/Tesla['Open'] 

<span class="vlist-t vlist-t2"><span class="vlist-r"><span class="vlist" style="height: 1.52594em;"><span class="" style="top: -3.52594em;"><span class="pstrut" style="height: 3.37144em;"></span><span class="mord"><span class="mord text"><span class="mord">Record&nbsp;High&nbsp;Percent</span></span><span class="mord rule" style="margin-right: 0.277778em;"></span><span class="mrel">=</span><span class="mord rule" style="margin-right: 0.277778em;"></span><span class="mord"><span class="mopen nulldelimiter"></span><span class="mfrac"><span class="vlist-t vlist-t2"><span class="vlist-r"><span class="vlist" style="height: 1.37144em;"><span class="" style="top: -2.314em;"><span class="pstrut" style="height: 3em;"></span><span class="mord"><span class="mord text"><span class="mord">New&nbsp;Highs</span></span><span class="mord rule" style="margin-right: 0.222222em;"></span><span class="mbin">+</span><span class="mord rule" style="margin-right: 0.222222em;"></span><span class="mord text"><span class="mord">New&nbsp;Lows</span></span></span></span><span class="" style="top: -3.15em;"><span class="pstrut" style="height: 3em;"></span><span class="stretchy" style="height: 0.2em;"><svg width="400em" height="0.2em" viewBox="0 0 400000 200" preserveAspectRatio="xMinYMin slice"><path d="M0 80H400000 v40H0z M0 80H400000 v40H0z"></path></svg></span></span><span class="" style="top: -3.677em;"><span class="pstrut" style="height: 3em;"></span><span class="mord"><span class="mord text"><span class="mord">New&nbsp;Highs</span></span></span></span></span><span class="vlist-s">​</span></span><span class="vlist-r"><span class="vlist" style="height: 0.88044em;"></span></span></span></span><span class="mclose nulldelimiter"></span></span><span class="mord rule" style="margin-right: 0.222222em;"></span><span class="mbin">×</span><span class="mord rule" style="margin-right: 0.222222em;"></span><span class="mord">1</span><span class="mord">0</span><span class="mord">0</span></span></span></span><span class="vlist-s">​</span></span><span class="vlist-r"><span class="vlist" style="height: 1.02594em;"></span></span></span>

https://www.investopedia.com/terms/h/high-low-index.asp

forecast_col = 'AdjClose'
forecast_out = int(math.ceil(0.01 * len(TeslaDF)))
TeslaDF['label'] = TeslaDF[forecast_col].shift(-forecast_out)


In [1]:
TeslaDF.head(3)

In [1]:
train_dataset = TeslaDF.sample(frac=0.8,random_state=0)
test_dataset = TeslaDF.drop(train_dataset.index)

In [1]:
train_stats = train_dataset.describe()
train_stats.pop("AdjClose")
train_stats = train_stats.transpose()
train_stats

In [1]:
train_labels = train_dataset.pop('AdjClose')
test_labels = test_dataset.pop('AdjClose')

In [1]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [1]:
def build_model():
    model = keras.Sequential([
      layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae', 'mse'])
    return model

In [1]:
model = build_model()

In [1]:
model.summary()


In [1]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

In [1]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [1]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [1]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("test_set average mas: {:5.2f} MPG".format(mae))

In [1]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

In [1]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [MPG]")
_ = plt.ylabel("Count")